#### bulk download World Bank Governance data

In [5]:
import pandas as pd
import numpy as np
import os
import time
import logging
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import openpyxl


In [12]:
#logger
logging.basicConfig(filename='WB governance.log', filemode='w', format='%(asctime)s - %(message)s', level=logging.INFO)

url='https://info.worldbank.org/governance/wgi/#home'

response=requests.get(url)
print(response.status_code)
#to get the url for download
soup=BeautifulSoup(response.content, 'html.parser')

#getting the href for Download full dataset (Excel) link in the webpage
s1=soup.find_all("ul", {"class": "listItems"})

url=s1[1].find_all("a", {"class": "file-download"})[2]['href']
domain='https://info.worldbank.org'
#construct the url
bulk_download_url=domain+url
#to download the url
try:
    r=requests.get(bulk_download_url)
    logging.info(f'Download request is sucessful {r.status_code}')
except Exception as e:
    logging.warning(e)

#write the raw data to excel
with open('WB governance.xlsx','wb') as f:
    try:
        f.write(r.content)
        logging.info('successfuly downloaded bulk excel file')
    except Exception as e:
        logging.warning(e)

200


In [36]:
wb = openpyxl.load_workbook("WB governance.xlsx")
print(wb.sheetnames)

['Introduction', 'VoiceandAccountability', 'Political StabilityNoViolence', 'GovernmentEffectiveness', 'RegulatoryQuality', 'RuleofLaw', 'ControlofCorruption']
